In [1]:
import pickle
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import tkhtmlview
import folium
import ipywidgets as widgets
import webbrowser
import cloudscraper
import json
from json import JSONDecodeError
import os.path

import warnings
warnings.filterwarnings('ignore')
import PySimpleGUI as sg

In [2]:
def predict(X):
    cb, X_train, y_train = pickle.load(open("Price_prediction_model.pkl", 'rb'))
    predictions = cb.predict(X)
    return predictions[0]


In [9]:
df=pd.read_csv('kc_house_data_fixed.csv')
df=df.drop(['Unnamed: 0','price'],axis=1)
# lat=47.3048
# lat2=lat+0.0005
# lat2=str(lat2)
# lat=str(lat)
# table_data=df.query(lat+'<lat<'+lat2)
# values = table_data.values.tolist()
# values
# #z=df.query( '47.3048<lat<47.3055')


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_basement,yr_built,yr_renovated,lat,long
23,3,2,1710,4697,1.5,0,0,5,6,0,1941,0,47.3048,-122.218


In [6]:
#this is the code of GUI app
def map_houses(long,lat):
    #print(coordinates)
    # create a map centered at the mean longitude and latitude of the coordinates
    mean_lat = sum(lat) / len(lat)
    mean_lon = sum(long) / len(long)
    house_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)
    coordinates = [list(a) for a in zip(lat, long)]
    # add a marker for each house to the map
    for coord in coordinates:
        folium.Marker(location=coord).add_to(house_map)
    
    # save the map to an HTML file
    house_map.save('house_map.html')
    
    # open the HTML file in a new tab
    webbrowser.open_new_tab('house_map.html')

def create_table(result_last, price_sqft_last,last_hous_atri):
    lat=float(last_hous_atri[-2])
    lat2=lat+0.0010
    lat2=str(lat2)
    lat=str(lat)
    table_data=df.query(lat+'<lat<'+lat2)
    table_data = table_data.values.tolist()
    for i in range(len(table_data)):
        result=predict([table_data[i]])
        price_sqft=round(result/table_data[i][2])
        result=round(result)
        table_data[i].insert(0,result)
        table_data[i].insert(1,price_sqft)
    table_data.insert(0,last_hous_atri)
    layout = [
    [sg.Column(layout = [[sg.Text('Table of House Prices:')],[sg.Table(values=table_data,
                                   headings=['price', 'Price/Sq.Ft','Bedrooms','Bathrooms','living-area','lot-area','floors','waterfront','view','condition',
                                                                'grade','basement-area','year_built','year_renovated','lat','long'],
                                   max_col_width=10, auto_size_columns=True,display_row_numbers=True,
                                  hide_vertical_scroll =True ,justification='center', size=(7, 18))]],
               scrollable=True,key='COL')],[sg.Button('Close',key='Close',size =(10, 1))]]
    window =sg.Window('House Price Table', layout,size=(1000, 350), finalize=True, resizable=True)
#    window['COL'].Widget.configure(borderwidth=1, relief=sg.SUNKEN, background_color='#D6D6D6')
    return window


def clear_data(table_window):
    with open('data.json', 'w') as f:
        json.dump([], f)
        if table_window is not None:
            table_window.close()
        window=create_table_window(0,0)
        return window

def save_data(data):
    if not os.path.exists('data.json'):
        with open('data.json', 'x') as f:
        # Write an empty list to the file
            json.dump([], f)
    with open('data.json', 'r') as f:
        try:
            saved_data = json.load(f)
        except JSONDecodeError:
            saved_data = []
    data = list(np.concatenate(data).flat)
    saved_data.append(data)
    
    with open('data.json', 'w') as f:
        json.dump(saved_data, f)


def retrieve_data():
    try:
        with open('data.json', 'r') as f:
            # Read the data from the file
            saved_data = json.load(f)
            return saved_data
    except FileNotFoundError:
        # If the file does not exist, create an empty list
        return []
    except JSONDecodeError:
        # If the file is empty, return an empty list
        return []
    
def create_table_window(result, price_sqft):
    table_data = retrieve_data()
    layout = [[sg.Text('Price of the house: {}'.format(result), key="new")],
              [sg.Text('Price/Sq.Ft: {}'.format(price_sqft), key="new")],
              [sg.Button('Show Locations', size=(14, 1),key='GPS', font=('Helvetica', 10),button_color=('white', '#3498db')),sg.Button('Nearby houses',key="neg", size=(14, 1), font=('Helvetica', 10),button_color=('white', '#3498db'))],
              #[sg.Button('Nerby Home Values',key="neg", size=(10, 1), button_color=('white', '#3498db'), font=('Helvetica', 12))],
    [sg.Column(layout = [[sg.Text('Table of House Prices:')],[sg.Table(values=table_data,
                                   headings=['price', 'Price/Sq.Ft','Bedrooms','Bathrooms','living-area','lot-area','floors','waterfront','view','condition',
                                                                'grade','basement-area','year_built','year_renovated','lat','long'],
                                   max_col_width=10, auto_size_columns=True,display_row_numbers=True,
                                  hide_vertical_scroll =True ,justification='center', size=(7, 18))]],
               scrollable=True,key='COL')],[sg.Button('Clear Table', key='clear_table'),sg.Button('Close',key='Close',size =(10, 1))]]
    window =sg.Window('House Price Table', layout,size=(1000, 350), finalize=True, resizable=True)
#    window['COL'].Widget.configure(borderwidth=1, relief=sg.SUNKEN, background_color='#D6D6D6')
    return window

def pridect_calc():
    layout = [[sg.Text('Enter the following details of your house to pridect Price:')],
              [sg.Text('No of bedrooms', size =(15, 1)), sg.Input(key='Bedrooms',size=(28, 1))],
              [sg.Text('No of bathrooms', size =(15, 1)), sg.Input(key='Bathrooms',size=(28, 1))],
              [sg.Text('Sqft living area', size =(15, 1)), sg.Input(key='living-area',size=(28, 1))],
              [sg.Text('Sqft lot area', size =(15, 1)), sg.Input(key='lot-area',size=(28, 1))],
              [sg.Text('No of floors', size =(15, 1)), sg.Input(key='floors',size=(28, 1))],
              [sg.Text('House waterfront', size =(15, 1)), sg.Checkbox("Yes",size=(11, 1),default=True)],
              [sg.Text('No of times viewed', size =(15, 1)), sg.Input(key='view',size=(28, 1))],
              [sg.Text('House Condition',size =(15, 1)),sg.Slider(range=(1, 5), default_value=1, orientation='h',size=(23.25, 15))],
              [sg.Text('House Grade',size =(15, 1)),sg.Slider(range=(3, 13), default_value=3, orientation='h',size=(23.25, 15))],
              [sg.Text('Sqft basement area', size =(15, 1)), sg.Input(key='basement-area',size=(28, 1))],
              [sg.Text('Year built', size =(15, 1)), sg.Input(key='year_built',size=(28, 1))],
              [sg.Text('Year renovated', size =(15, 1)), sg.Input(key='year_renovated',size=(28, 1))],
              [sg.Text('Lateral coordinate', size =(15, 1)), sg.Input(key='lat',size=(28, 1))],
              [sg.Text('Longitude coordinate', size =(15, 1)), sg.Input(key='long',size=(28, 1))],
              [sg.HorizontalSeparator()],
              [sg.Column(layout=[[sg.Button('Pridect',size =(10, 1),button_color=('white', '#3498db')), 
                                 sg.Button('Clear',size =(10, 1),button_color=('white', '#3498db')),sg.Button('Houses table',key='house_table',size =(10, 1),button_color=('white', '#3498db'))]],justification='center')]
             ]
    window = sg.Window('price house prediction',layout, resizable=True)
    while True:
        event, values = window.read()
        try:
            inputs = [i for i in values.values()]
            keys_to_clear=[i for i in values.keys()]
        except AttributeError:
            pass
        if values and values['living-area'] != '' and values['living-area'] is not None:
            try:
                sqft_liv = float(values['living-area'])
            except ValueError:
                window['living-area'].update('')
                window['living-area'].set_focus()
        if event == sg.WIN_CLOSED:
            break
        if event == 'Pridect':
            # Validate the input
            invalid = False
            for key in values:
                try:
                    float(values[key])
                except ValueError:
                    invalid = True
                    # Clear the input field and set the focus to it
                    window[key].update('')
                    window[key].set_focus()
                    break
            if invalid:
                sg.popup_ok('Please Enter valid numbers:', auto_close=True, auto_close_duration=3)
            else:
                result=predict([inputs])
                price_sqft=round(result/sqft_liv)
                result=round(result)
                window.close()
                inputs.insert(0,result)
                inputs.insert(1,price_sqft)
                save_data([inputs])
                table_window = create_table_window(result,price_sqft)
                while True:
                    event, values = table_window.read()
                    if event =='Close' or event == sg.WIN_CLOSED:
                        table_window.close()
                        pridect_calc()
                        break
                    if event == 'clear_table':
                        table_window =clear_data(table_window)
                    if event == 'neg':
                        table_window.close()
                        table_window2 = create_table(result,price_sqft,inputs)
                        while True:
                            event, values = table_window2.read()
                            if event =='Close' or event == sg.WIN_CLOSED:
                                table_window2.close()
                                table_window =create_table_window(result,price_sqft)
                                break
                    if event == 'GPS':
                        my_list=retrieve_data()
                        long=[float(item[-1]) for item in my_list]
                        lat=[float(item[-2]) for item in my_list]
                        #table_window.close()
                        table_window2 = map_houses(long,lat)                        
                break
        if event=='house_table':
            my_list=retrieve_data()
            if my_list != []:
                x=len(my_list)-1
                inputs=my_list[x]
                result=my_list[x][0]
                price_sqft=my_list[x][1]
            else:
                inputs=[]
                result=0
                price_sqft=0
            table_window = create_table_window(result,price_sqft)
            window.close()
            while True:
                event, values = table_window.read()
                if event =='Close' or event == sg.WIN_CLOSED:
                    table_window.close()
                    pridect_calc()
                    break
                if event == 'clear_table':
                    if my_list != []:
                        table_window=clear_data(table_window)
                    else:
                        sg.popup_ok('You didn\'t predict a houses ', auto_close=True, auto_close_duration=3)
                        window.close()
                        table_window.close()
                        pridect_calc()
                if event == 'neg':
                    if my_list == []:
                        sg.popup_ok('You didn\'t predict a houses ', auto_close=True, auto_close_duration=3)
                        window.close()
                        table_window.close()
                        pridect_calc()
                        break
                    else:
                        table_window.close()
                        table_window2 = create_table(result,price_sqft,inputs)
                        while True:
                            event, values = table_window2.read()
                            if event =='Close' or event == sg.WIN_CLOSED:
                                table_window2.close()
                                table_window =create_table_window(result,price_sqft)
                                break
                if event == 'GPS':
                    if my_list != []:
                        my_list=retrieve_data()
                        long=[float(item[-1]) for item in my_list]
                        lat=[float(item[-2]) for item in my_list]
                        #table_window.close()
                        table_window2 = map_houses(long,lat)
                    else:
                        sg.popup_ok('You didn\'t predict a houses ', auto_close=True, auto_close_duration=3)
                        window.close()
                        table_window.close()
                        pridect_calc()
            break
        if event == 'Clear':
             for key in values:
                if values[key] != '':
                    if values[key] == '':
                        invalid = True
                        window[key].update('')
                        window[key].set_focus()
                        break
                    else:
                        try:
                            float(values[key])

                        except ValueError:
                            invalid = True
                            window[key].update('')
                            window[key].set_focus()
                            break
                else:
                    window[key].update('')
                    window[key].set_focus()
                    break
def main():
    # Get the image from the URL
    url = "https://kingcounty.gov/img/horiz_transbg.gif"
    jpg_data = (
        cloudscraper.create_scraper(
            browser={"browser": "firefox", "platform": "windows", "mobile": False}
        ).get(url).content
    )

    # Create the layout for the window
    layout = [
        [
            sg.Column(
                layout=[
                    [sg.Image(data=jpg_data, key="-ArtistAvatarIMG-")],
                    [sg.Text('Welcome to \n Kings County House Price Predictor!', font=('Helvetica', 24), text_color='#3498db', justification='center')],
                    [sg.Text('Are you ready to find the perfect house?', font=('Helvetica', 14), text_color='#808080')],
                    [sg.Button('Start', size=(10, 1), button_color=('white', '#3498db'), font=('Helvetica', 12))]
                ],
                element_justification='center',
            )
        ]
    ]

    # Create the window
    window = sg.Window('Welcome', layout, modal=True)

    # Loop until the user clicks the 'Start' button or closes the window
    while True:
        event, values = window.read()
        if event == 'Start':
            window.close()
            pridect_calc()
        else:
            if event == "Cancel" or event == sg.WIN_CLOSED:
                break

    # Close the window
    window.close()
if __name__ == "__main__":
    sg.set_options(font=("Didot", 12))
    sg.change_look_and_feel('DarkBlue2')
    #sg.layout,size=(800,600)
    main()

In [5]:
#[["226245", "192", "3", "1", "1180", "5650", "1", "False", "0", "3.0", "7.0", "0", "1955", "0", "47.5112", "-122.257"], ["547137", "213", "3", "2", "2570", "7242", "2", "False", "0", "3.0", "7.0", "2170", "1951", "1991", "47.7210", "-122.319"], ["242582", "315", "2", "1", "770", "10000", "1", "False", "0", "3.0", "6.0", "0", "1933", "0", "47.7379", "-122.233"], ["546436", "279", "4", "3", "1960", "5000", "1", "False", "0", "3.0", "8.0", "0", "1987", "0", "47.6168", "-122.045"], ["356286", "154", "4", "2", "2310", "5813", "2", "False", "0", "3.0", "7.0", "0", "2009", "0", "47.51107", "-122.299"], ["245824", "241", "2", "1", "1020", "1076", "2", "False", "0", "3.0", "7.0", "0", "2008", "0", "47.5941", "-122.299"]]